# Create time dimension

In [0]:
min_date_qry = spark.sql('''
    SELECT MIN(started_at) as started_at FROM trips
''')
min_date = min_date_qry.first().asDict()['started_at']

max_date_qry = spark.sql('''
    SELECT DATEADD(year, 5, MAX(started_at)) as started_at FROM trips
''')
max_date = max_date_qry.first().asDict()['started_at']

expression = f"sequence(to_date('{min_date}'), to_date('{max_date}'), interval 1 day)"

In [0]:
from pyspark.sql import functions as f


dim_time = spark.createDataFrame([(1,)], ["time_id"])

dim_time = dim_time.withColumn("dateinit", f.explode(f.expr(expression)))
dim_time = dim_time.withColumn("date", f.to_timestamp(dim_time.dateinit, "yyyy-MM-dd"))
dim_time = dim_time \
    .withColumn("year", f.year(dim_time.date)) \
    .withColumn("quarter", f.quarter(dim_time.date)) \
    .withColumn("month", f.month(dim_time.date)) \
    .withColumn("day", f.dayofweek(dim_time.date)) \
    .withColumn("time_id", dim_time.date.cast(StringType())) \
    .drop(col("dateinit"))

dim_time.show(5)

+-------------------+-------------------+----+-------+-----+---+
 time_id| date|year|quarter|month|day|
+-------------------+-------------------+----+-------+-----+---+
2021-02-01 00:00:00|2021-02-01 00:00:00|2021| 1| 2| 2|
2021-02-02 00:00:00|2021-02-02 00:00:00|2021| 1| 2| 3|
2021-02-03 00:00:00|2021-02-03 00:00:00|2021| 1| 2| 4|
2021-02-04 00:00:00|2021-02-04 00:00:00|2021| 1| 2| 5|
2021-02-05 00:00:00|2021-02-05 00:00:00|2021| 1| 2| 6|
+-------------------+-------------------+----+-------+-----+---+
only showing top 5 rows

In [0]:
dim_time.agg({'year': 'max'}).show()

+---------+
max(year)|
+---------+
 2027|
+---------+

In [0]:
dim_time.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_time")

In [0]:
%sql
select * from dim_time limit 5

time_id,date,year,quarter,month,day
2021-02-01 00:00:00,2021-02-01T00:00:00.000+0000,2021,1,2,2
2021-02-02 00:00:00,2021-02-02T00:00:00.000+0000,2021,1,2,3
2021-02-03 00:00:00,2021-02-03T00:00:00.000+0000,2021,1,2,4
2021-02-04 00:00:00,2021-02-04T00:00:00.000+0000,2021,1,2,5
2021-02-05 00:00:00,2021-02-05T00:00:00.000+0000,2021,1,2,6


In [0]:
dim_time.printSchema()


root
-- time_id: string (nullable = true)
-- date: timestamp (nullable = true)
-- year: integer (nullable = true)
-- quarter: integer (nullable = true)
-- month: integer (nullable = true)
-- day: integer (nullable = true)

# create payment fact table

In [0]:
fact_payment = spark.sql('''
    SELECT
        payment_id,
        payments.amount,
        payments.rider_id,
        dim_time.time_id
    FROM payments
    JOIN dim_time ON dim_time.date = payments.date
''')

In [0]:
fact_payment.show()

+----------+------+--------+-------------------+
payment_id|amount|rider_id| time_id|
+----------+------+--------+-------------------+
 22| 9.0| 1000|2021-02-01 00:00:00|
 23| 9.0| 1000|2021-03-01 00:00:00|
 24| 9.0| 1000|2021-04-01 00:00:00|
 25| 9.0| 1000|2021-05-01 00:00:00|
 26| 9.0| 1000|2021-06-01 00:00:00|
 27| 9.0| 1000|2021-07-01 00:00:00|
 28| 9.0| 1000|2021-08-01 00:00:00|
 29| 9.0| 1000|2021-09-01 00:00:00|
 30| 9.0| 1000|2021-10-01 00:00:00|
 31| 9.0| 1000|2021-11-01 00:00:00|
 32| 9.0| 1000|2021-12-01 00:00:00|
 33| 9.0| 1000|2022-01-01 00:00:00|
 34| 9.0| 1000|2022-02-01 00:00:00|
 62| 6.56| 1003|2021-02-01 00:00:00|
 63| 7.4| 1003|2021-03-01 00:00:00|
 64| 18.31| 1003|2021-04-01 00:00:00|
 65| 14.48| 1003|2021-05-01 00:00:00|
 66| 6.5| 1003|2021-06-01 00:00:00|
 67| 12.35| 1003|2021-07-01 00:00:00|
 68| 22.84| 1003|2021-08-01 00:00:00|
+----------+------+--------+-------------------+
only showing top 20 rows

In [0]:
fact_payment.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_payment")

In [0]:
%sql
select * from fact_payment limit 5

payment_id,amount,rider_id,time_id
22,9.0,1000,2021-02-01 00:00:00
23,9.0,1000,2021-03-01 00:00:00
24,9.0,1000,2021-04-01 00:00:00
25,9.0,1000,2021-05-01 00:00:00
26,9.0,1000,2021-06-01 00:00:00


# create trip fact

In [0]:
fact_trip = spark.sql('''
    SELECT 
        trips.trip_id,
        riders.rider_id,
        trips.start_station_id, 
        trips.end_station_id, 
        start_time.time_id                                                  AS start_time_id,
        end_time.time_id                                                    AS end_time_id,
        trips.rideable_type,
        DATEDIFF(hour, trips.started_at, trips.ended_at)                    AS duration,
        DATEDIFF(year, riders.birthday, trips.started_at)                     AS rider_age

    FROM trips
    JOIN riders                     ON riders.rider_id = trips.rider_id
    JOIN dim_time AS start_time     ON start_time.date = trips.started_at
    JOIN dim_time AS end_time       ON end_time.date = trips.ended_at
''')

In [0]:
fact_trip.show()

+-------+--------+----------------+--------------+-------------+-----------+-------------+--------+---------+
trip_id|rider_id|start_station_id|end_station_id|start_time_id|end_time_id|rideable_type|duration|rider_age|
+-------+--------+----------------+--------------+-------------+-----------+-------------+--------+---------+
+-------+--------+----------------+--------------+-------------+-----------+-------------+--------+---------+

In [0]:
fact_trip.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("fact_trip")

In [0]:
%sql
SELECT * from fact_trip limit 2

trip_id,rider_id,start_station_id,end_station_id,start_time_id,end_time_id,rideable_type,duration,rider_age


In [0]:
%sql
SELECT * from dim_time limit 2

time_id,date,year,quarter,month,day
2021-02-01 00:00:00,2021-02-01T00:00:00.000+0000,2021,1,2,2
2021-02-02 00:00:00,2021-02-02T00:00:00.000+0000,2021,1,2,3


# create rider dimension

In [0]:
dim_rider = spark.sql('''
    SELECT 
        rider_id, firstName, lastName, address, birthday, startDate, endDate, isMember
    FROM riders
''')

In [0]:
dim_rider.show(2)

+--------+---------+--------+-------------------+-------------------+-------------------+-------------------+--------+
rider_id|firstName|lastName| address| birthday| startDate| endDate|isMember|
+--------+---------+--------+-------------------+-------------------+-------------------+-------------------+--------+
 1000| Diana| Clark|1200 Alyssa Squares|1989-02-13 00:00:00|2019-04-23 00:00:00| null| true|
 1001| Jennifer| Smith| 397 Diana Ferry|1976-08-10 00:00:00|2019-11-01 00:00:00|2020-09-01 00:00:00| true|
+--------+---------+--------+-------------------+-------------------+-------------------+-------------------+--------+
only showing top 2 rows

In [0]:
dim_rider.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_rider")

In [0]:
%sql
SELECT * from dim_rider limit 2

rider_id,firstName,lastName,address,birthday,startDate,endDate,isMember
1000,Diana,Clark,1200 Alyssa Squares,1989-02-13T00:00:00.000+0000,2019-04-23T00:00:00.000+0000,null,true
1001,Jennifer,Smith,397 Diana Ferry,1976-08-10T00:00:00.000+0000,2019-11-01T00:00:00.000+0000,2020-09-01T00:00:00.000+0000,true


# create station dimension

In [0]:
dim_station = spark.sql('''
    SELECT station_id, name, latitude , longitude FROM stations
''')

In [0]:
dim_station.show(2)

+------------+--------------------+-----------------+------------------+
 station_id| name| latitude| longitude|
+------------+--------------------+-----------------+------------------+
 525|Glenwood Ave & To...| 42.012701|-87.66605799999999|
KA1503000012| Clark St & Lake St|41.88579466666667|-87.63110066666668|
+------------+--------------------+-----------------+------------------+
only showing top 2 rows

In [0]:
dim_station.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_station")

In [0]:
%sql
SELECT * from dim_station limit 2

station_id,name,latitude,longitude
525,Glenwood Ave & Touhy Ave,42.012701,-87.66605799999999
KA1503000012,Clark St & Lake St,41.88579466666667,-87.63110066666668


# create payment dimension

In [0]:
dim_payment = spark.sql('''
    SELECT
        payment_id,
        payments.amount,
        payments.rider_id,
        dim_time.time_id
    FROM payments
    JOIN dim_time ON dim_time.date = payments.date
''')

In [0]:
dim_payment.show(2)

+----------+------+--------+-------------------+
payment_id|amount|rider_id| time_id|
+----------+------+--------+-------------------+
 22| 9.0| 1000|2021-02-01 00:00:00|
 23| 9.0| 1000|2021-03-01 00:00:00|
+----------+------+--------+-------------------+
only showing top 2 rows

In [0]:
dim_payment.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("dim_payment")

In [0]:
%sql
SELECT * from dim_payment limit 2

payment_id,amount,rider_id,time_id
22,9.0,1000,2021-02-01 00:00:00
23,9.0,1000,2021-03-01 00:00:00
